In [1]:
import os
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score, precision_score, recall_score, f1_score

# Download NLTK resources (only needs to be done once)
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /Users/i538819/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/i538819/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Step 1: Basic preprocessing

In [2]:
def simple_preprocess(text):
    if not isinstance(text, str):
        text = ""
    return word_tokenize(text.lower())

## Step 2: Compute average word embedding for a document

In [3]:
def average_embedding(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

## Step 3: Compute cosine similarity between two vectors

In [4]:
def compute_similarity(vec1, vec2):
    return cosine_similarity([vec1], [vec2])[0][0]

## Step 4: Convert similarity score to class label (0 to 3)

In [5]:
def classify_similarity(sim):
    if sim >= 0.90:
        return 4
    elif sim >= 0.75:
        return 3
    elif sim >= 0.50:
        return 2
    else:
        return 1

## Main comparison function

In [6]:
def compare_articles(text_a, text_b, model):
    tokens_a = simple_preprocess(text_a)
    tokens_b = simple_preprocess(text_b)

    vec_a = average_embedding(tokens_a, model)
    vec_b = average_embedding(tokens_b, model)

    sim = compute_similarity(vec_a, vec_b)
    label = classify_similarity(sim)

    return sim, label

## Classification Prediction

In [7]:
def predict_classification_with_titles(X, model):
    predictions = []

    for _, row in X.iterrows():
        title_a = row['title1']
        title_b = row['title2']

        sim, label = compare_articles(title_a, title_b, model)
        predictions.append(label)

    return predictions

def predict_classification_with_text(X, model):
    predictions = []

    for _, row in X.iterrows():
        text_a = row['text1']
        text_b = row['text2']

        sim, label = compare_articles(text_a, text_b, model)
        predictions.append(label)

    return predictions

## Regression Prediction

In [8]:
def predict_regression_with_titles(X, model):
    predictions = []

    for _, row in X.iterrows():
        title_a = row['title1']
        title_b = row['title2']

        sim, label = compare_articles(title_a, title_b, model)
        predictions.append(sim)

    return predictions

## Main Evaluation Function

In [9]:
def evaluate(data, model):
    # Predict as Classification
    print("\nClassification using only titles...")
    data['pred_class_title_only'] = predict_classification_with_titles(data, model)
    acc = accuracy_score(data['overall_classification'], data['pred_class_title_only'])
    prec = precision_score(data['overall_classification'], data['pred_class_title_only'], average='weighted')
    rec = recall_score(data['overall_classification'], data['pred_class_title_only'], average='weighted')
    f1 = f1_score(data['overall_classification'], data['pred_class_title_only'], average='weighted')
    print(f"Accuracy Classification (titles only): {acc:.3f}")
    print(f"Precision Classification (titles only): {prec:.3f}")
    print(f"Recall Classification (titles only): {rec:.3f}")
    print(f"F1 Score Classification (titles only): {f1:.3f}")

    print("\nClassification using only text...")
    data['pred_class_text_only'] = predict_classification_with_text(data, model)
    acc = accuracy_score(data['overall_classification'], data['pred_class_text_only'])
    prec = precision_score(data['overall_classification'], data['pred_class_text_only'], average='weighted')
    rec = recall_score(data['overall_classification'], data['pred_class_text_only'], average='weighted')
    f1 = f1_score(data['overall_classification'], data['pred_class_text_only'], average='weighted')
    print(f"Accuracy Classification (text only): {acc:.3f}")
    print(f"Precision Classification (text only): {prec:.3f}")
    print(f"Recall Classification (text only): {rec:.3f}")
    print(f"F1 Score Classification (text only): {f1:.3f}")

    # Predict as Regression
    print("\nRegression using only titles...")
    data['pred_reg_title_only'] = predict_regression_with_titles(data, model)
    # Scale from [0, 1] to [1, 4] range
    data['pred_reg_title_only'] = data['pred_reg_title_only'] * 3 + 1

    mse = mean_squared_error(data['overall'], data['pred_reg_title_only'])
    mae = mean_absolute_error(data['overall'], data['pred_reg_title_only'])
    r2 = r2_score(data['overall'], data['pred_reg_title_only'])

    print("Scores Regression (titles only):")
    print(f"MSE: {mse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R² : {r2:.4f}")

    print()
    display(data)

## Evaluation with Word2Vec

In [10]:
# Load CSV with all pairs
print("Loading CSV...")
df = pd.read_csv("data/full_dataset.csv")

# Filter for English-only pairs
df = df[(df["lang1"] == "en") & (df["lang2"] == "en")].reset_index(drop=True)

# Load pre-trained Word2Vec model (Google News vectors)
print("Loading Word2Vec model ...")
model = KeyedVectors.load_word2vec_format("pretrained_models/word2vec/GoogleNews-vectors-negative300.bin", binary=True)

evaluate(df, model)

Loading CSV...
Loading Word2Vec model ...

Classification using only titles...
Accuracy Classification (titles only): 0.139
Precision Classification (titles only): 0.133
Recall Classification (titles only): 0.139
F1 Score Classification (titles only): 0.098

Classification using only text...
Accuracy Classification (text only): 0.160
Precision Classification (text only): 0.144
Recall Classification (text only): 0.160
F1 Score Classification (text only): 0.138

Regression using only titles...
Scores Regression (titles only):
MSE: 2.6270
MAE: 1.4417
R² : -1.2483



,pair_id,title1,text1,lang1,title2,text2,lang2,overall,overall_classification,geography,entities,time,narrative,style,tone,pred_class_title_only,pred_class_text_only,pred_reg_title_only
0,1484084337_1484110209,Virginia man arrested in fatal DUI crash in We...,"MARTINSBURG, W.Va. — A suspected drunken drive...",en,Haiti’s leader marks independence day amid sec...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",en,4.000000,4,4.0,4.000000,1.000000,4.000000,1.666667,2.000000,1,3,1.942115
1,1484396422_1483924666,Guyana: Three injured after car crashes into u...,Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,en,Fire kills more than 30 animals at zoo in west...,BERLIN - A fire at a zoo in western Germany in...,en,3.666667,4,4.0,4.000000,1.000000,4.000000,1.666667,1.333333,1,3,2.419770
2,1484698254_1483758694,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,(Breitbart) – President Donald Trump welcomed ...,en,"Trump says he does not expect war with Iran, ‘...","PALM BEACH, United States — US President Donal...",en,2.333333,2,1.0,2.000000,1.000000,2.333333,1.000000,1.333333,2,4,2.954722
3,1576314516_1576455088,Zomato Buys Uber's Food Delivery Business in I...,Uber has sold its online food-ordering busines...,en,Indian Online Food Delivery Market to Hit $8 B...,Rapid digitisation and growth in both online b...,en,2.000000,2,1.0,2.333333,2.666667,1.666667,1.666667,1.666667,2,4,3.076404
4,1484036253_1483894099,"India approves third moon mission, months afte...",BENGALURU (Reuters) - India has approved its t...,en,India targets new moon mission in 2020,BANGALORE: India plans to make a fresh attempt...,en,1.250000,1,1.0,1.250000,1.000000,1.250000,1.000000,1.000000,2,4,3.223975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,1484430020_1483808942,Mawela refutes claims that police did not move...,Police have disputed reports of a third death ...,en,"Two killed, several wounded in Melville drive-...",JOHANNESBURG - A manhunt is underway after two...,en,1.000000,1,1.0,1.000000,1.000000,1.000000,2.000000,1.000000,1,4,2.431473
1662,1562189733_1562214022,Lockdown in seven Hulu Langat villages after t...,Soldiers and police officers conducting checks...,en,"Batu 21-24 in Hulu Langat under enhanced MCO, ...","29/03/2020 08:15 PM\n\nKUALA LUMPUR, March 29 ...",en,1.000000,1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1,4,2.441467
1663,1490711494_1500549122,Iran tensions: Soleimani killing does not make...,The Trump administration’s killing of Iranian ...,en,World record attempt for largest gathering of ...,Sri Lanka’s attempt at setting a new world rec...,en,4.000000,4,4.0,4.000000,2.000000,4.000000,4.000000,4.000000,1,3,2.326559
1664,1559851360_1553590552,Distilleries producing hand sanitizer to help ...,With so many businesses forced to temporarily ...,en,Triad distillery using its facility to make ha...,Hide Transcript Show Transcript\n\nNEWS. KENNY...,en,1.000000,1,2.0,2.000000,1.000000,1.000000,1.000000,1.000000,2,4,2.786041


## Evaluation with FastText

In [11]:
# Load CSV with all pairs
print("Loading CSV...")
df2 = pd.read_csv("data/full_dataset.csv")

# Load FastText word vectors
print("Loading FastText model...")
if os.path.exists("pretrained_models/fasttext/wiki-news-300d-1M-subword.kv"):
    model2 = KeyedVectors.load("pretrained_models/fasttext/wiki-news-300d-1M-subword.kv")  # faster loading
else:
    model2 = KeyedVectors.load_word2vec_format("pretrained_models/fasttext/wiki-news-300d-1M-subword.vec")
    model2.save("pretrained_models/fasttext/wiki-news-300d-1M-subword.kv")

evaluate(df2, model2)

Loading CSV...
Loading FastText model...

Classification using only titles...
Accuracy Classification (titles only): 0.181
Precision Classification (titles only): 0.203
Recall Classification (titles only): 0.181
F1 Score Classification (titles only): 0.162

Classification using only text...
Accuracy Classification (text only): 0.379
Precision Classification (text only): 0.260
Recall Classification (text only): 0.379
F1 Score Classification (text only): 0.259

Regression using only titles...
Scores Regression (titles only):
MSE: 2.0741
MAE: 1.1981
R² : -0.6321



,pair_id,title1,text1,lang1,title2,text2,lang2,overall,overall_classification,geography,entities,time,narrative,style,tone,pred_class_title_only,pred_class_text_only,pred_reg_title_only
0,1484084337_1484110209,Virginia man arrested in fatal DUI crash in We...,"MARTINSBURG, W.Va. — A suspected drunken drive...",en,Haiti’s leader marks independence day amid sec...,"PORT-AU-PRINCE, Haiti — Haitian President Jove...",en,4.000000,4,4.0,4.000000,1.000000,4.000000,1.666667,2.000000,1,4,1.599963
1,1484396422_1483924666,Guyana: Three injured after car crashes into u...,Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,en,Fire kills more than 30 animals at zoo in west...,BERLIN - A fire at a zoo in western Germany in...,en,3.666667,4,4.0,4.000000,1.000000,4.000000,1.666667,1.333333,2,4,2.939217
2,1484698254_1483758694,Trump Brings In 2020 At Mar-a-Lago: ‘We’re Goi...,(Breitbart) – President Donald Trump welcomed ...,en,"Trump says he does not expect war with Iran, ‘...","PALM BEACH, United States — US President Donal...",en,2.333333,2,1.0,2.000000,1.000000,2.333333,1.000000,1.333333,4,4,3.795657
3,1576314516_1576455088,Zomato Buys Uber's Food Delivery Business in I...,Uber has sold its online food-ordering busines...,en,Indian Online Food Delivery Market to Hit $8 B...,Rapid digitisation and growth in both online b...,en,2.000000,2,1.0,2.333333,2.666667,1.666667,1.666667,1.666667,1,4,1.891695
4,1484036253_1483894099,"India approves third moon mission, months afte...",BENGALURU (Reuters) - India has approved its t...,en,India targets new moon mission in 2020,BANGALORE: India plans to make a fresh attempt...,en,1.250000,1,1.0,1.250000,1.000000,1.250000,1.000000,1.000000,2,4,3.042312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,1504517935_1504699849,"العثور على طفلة رضيعة داخل كرتونه بـ""المنوفية""","العثور على طفلة رضيعة داخل كرتونه بـ""المنوفية""...",ar,العثور على طفل داخل كرتونة في الشارع بالمنوفية,عثر أهالي مدينة الباجور بمحافظة المنوفية علي ط...,ar,1.000000,1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,4,4,3.868175
2732,1612662952_1575561189,القصة الكاملة لمعاناة أسرة من تنمر ومنع طعام و...,نشرت صفحة أطفال مفقودة على فيسبوك في مصر وهي ص...,ar,وكالة أنباء اليسار,التبرع للموقع - ادعمونا\n\n\n\n\n\nالموقع الرئ...,ar,3.000000,3,3.0,3.000000,3.000000,3.000000,1.000000,1.000000,1,4,1.000000
2733,1610474240_1502834406,دوي انفجارات عنيفة في العاصمة صنعاء الأماكن ال...,دوي انفجارات عنيفة في العاصمة صنعاء الأماكن ال...,ar,مصرع 15 حوثياً في حريب نهم شرق صنعاء,قالت مصادر عسكرية إن 15 عنصراً من مليشيا الحوث...,ar,2.000000,2,1.0,1.000000,4.000000,2.000000,1.000000,1.000000,3,4,3.646939
2734,1548303752_1548337846,الأرصاد تكشف عن أحوال الطقس حتى نهاية فصل الشت...,قال محمود شاهين مدير مركز التحاليل والتنبؤات ا...,ar,صوت المسيحى الحر,NaN,ar,1.000000,1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,3,1,3.338609
